In [1]:
import numpy as np
import pandas as pd
import sklearn 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from fuzzywuzzy import process
import re
import spacy
import operator
from fuzzywuzzy import fuzz
import operator
from collections import OrderedDict
import time

C:\Users\mayan\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
intity_design = pd.read_json("doc10.json")
intity_design.head()

,intent_model
0,"{'context': 'Doctor', 'utterances': ['Who are ..."
1,"{'context': 'Patient', 'utterances': ['what is..."


In [3]:
data_frame = pd.json_normalize(intity_design["intent_model"])
data_frame.head()

,context,utterances
0,Doctor,"[Who are the best doctors for heart disease, W..."
1,Patient,[what is the minium age of the patient of doct...


In [4]:
data_frame["utterances"][0] = [i.lower() for i in data_frame["utterances"][0] ]
data_frame["utterances"][1] = [j.lower() for j in data_frame["utterances"][1]]


### Context understanding 

In [5]:
class clean:
    def lowerlettercleanword(sentence):
        sentences = [word for sent in sentence for word in sent]
        return sentences

In [6]:
wx = clean.lowerlettercleanword(data_frame["utterances"])
print(wx)


['who are the best doctors for heart disease', 'who are the best doctors for cardiovascular disease', 'who are the best doctors for skin', 'best doctors for dermatology', 'best physician', 'what is the clinic location of doctor mayank', 'best doctor near my location for cardiology', 'please provide the name of the doctors which have fees less than 500 rupees', 'what is timing for doctor soumya', 'which is the best dental clinic', 'what is the dr. rishabh consutant fees', 'doctor with consultant fees more than 1000 rupees', 'what is the speciliatity of doctor piyush', 'what is the speciliatity of doctor mayank', 'what is the speciliatity of doctor soumya', 'what is the speciliatity of doctor rishabh', 'what is the speciliatity of doctor deepshikha', 'what is the clinic location of doctor piyush', 'what is the clinic location of doctor soumya', 'what is the clinic location of doctor deepshikha', 'what is consultant fees of doctor piyush', 'what is consultant fees of doctor mayank', 'what

In [7]:
def getContext(match_string):
    match_string = match_string.lower()
    for idx, i in enumerate(data_frame['utterances']):
        if match_string in i:
            return data_frame['context'][idx]
    return None

In [8]:
print(getContext('Who are the best doctors for skin'))

Doctor


_____________________________________________________________

### Name Entity extraction

In [9]:
nlp = spacy.load('en_core_web_sm')

In [10]:
def get_name(doc):
    if doc.ents:
        for ents in doc.ents:
            if ents.label_=='PERSON':
                name = ents.text
                return name
            return None
    else:
        return None
            

In [11]:
doctors =["Piyush Singh","Soumya","Mayank Mishra","Deepshikha Mishra","Rishabh Yadav","Piyush","Soumya","Mayank","Deepshikha","piyush","soumya","mayank","deepshikha","rishabh"]

def get_name1(query):
    query = query.split()
    for i in query:
        for j in doctors:
            if i==j:
                name = j
                return name
            

In [12]:
print(get_name(nlp("Jeff Bezos is the CEO of Amazon")))

Jeff Bezos


In [13]:
print(get_name(nlp("Doctor Soumya  Speiliaty")))

Soumya


In [14]:
print(get_name(nlp("Doctor Rishabh Yadav  Speiliaty")))

Rishabh Yadav


In [15]:
print(get_name(nlp("Edward Makahaya Clinic Location")))

Edward Makahaya


____________________________________________

### Best recommendation and exact match string

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
stopwords_list = stopwords.words('english')
vectorizer = TfidfVectorizer(max_features=3000, stop_words=stopwords_list)
X = vectorizer.fit_transform(wx)
count_dense = X.todense()


In [18]:
test_query = vectorizer.transform(["Best cardiologist"])
cosine_similarity(test_query, count_dense[1,:])[0][0]

0.42002790503955745

In [19]:

def getContext_details(input_query):
    input_query_vector = vectorizer.transform([input_query])
    max_score = 0
    best_match = None
    context = None
    for ind, vector in enumerate(count_dense):
        sim_score = cosine_similarity(input_query_vector, count_dense[ind,:])[0][0]
        if sim_score>=max_score:
            max_score = sim_score
            best_match = wx[ind]
            context = getContext(best_match)
    return max_score, best_match, context


In [20]:
print(getContext_details("Cardiologist clinic location near me "))

(0.6049548668886633, 'best doctor near my location for cardiology', 'Doctor')


In [21]:
print(getContext_details("what disease is garima suffering from"))

(0.7918772370034812, 'disease name of patient garima', 'Patient')


In [22]:
print(getContext_details("What is the cause of death of Aditya Sonkar"))

(0.7364491630507464, 'what is the health issue with aditya sonkar', 'Patient')


In [23]:
print(getContext_details("Doctor Mayank clinic location"))

(1.0, 'what is the clinic location of doctor mayank', 'Doctor')


In [24]:

print(getContext_details("what is the consultant fees of doctor soumya"))

(1.0, 'what is the consultant fees of doctor soumya', 'Doctor')


In [25]:
def patient_name(query):
    name = None
    if query == 1:
        name = "Aditya Sonkar"
    elif query == 2:
        name = "Harsh Mishra"
    elif query == 3:
        name = "Yuvraj Sinha"
    elif query == 4:
        name = "Amit"
    elif query == 5:
        name = "Garima"
    else:
        name = "Patient id is Wrong"
    return name    

In [26]:
def regrex(query):
    for i in query:
        query = re.sub("dr","doctor",query)
        query = re.sub("Dr","doctor",query)
        query = re.sub("DR","doctor",query)
        query = re.sub("disease","health issue",query)
        query = re.sub("sickness","health issue",query)
        query = re.sub("illness","health issue",query)
        query = re.sub("infection","health issue",query)
        query = re.sub("Illness","health issue",query)
        query =  re.sub("Disease","health issue",query)
        query = re.sub("Sickess","health issue",query)
        query = re.sub("Charges","fees",query)
        query = re.sub("charges","fees",query)
        query = re.sub("wage","fees",query)
        query = re.sub("Wages","fees",query)
        query = re.sub(r"won\'t", "will not", query)
        query = re.sub(r"can\'t","can not",query)
        query = re.sub(r"n\'t", "not", query)
        query= re.sub(r"\'re", "are", query)
        query = re.sub(r"\'s", "is", query)
        query = re.sub(r"\'d", "would", query)
        query = re.sub(r"\'ll", "will", query)
        query = re.sub(r"\'t", "not", query)
        query = re.sub(r"\'ve", "have", query)
        query = re.sub(r"\'m", "am", query)
        query= re.sub("[?]+","",query)
        
    return query    

In [27]:
class bestrecommend:
    def getContext_bestrecommend(input_query):
        input_query = input_query.lower()
        input_query_vector = vectorizer.transform([input_query])
        unsorted_score = []
        unsorted_match = []

        for ind, vector in enumerate(count_dense):
            sim_score = cosine_similarity(input_query_vector, count_dense[ind,:])[0][0]
            if sim_score>=0.4:
                unsorted_score.append(sim_score)
                unsorted_match.append(wx[ind])
        zipped_pairs = zip(unsorted_score,unsorted_match)
        best_recom = [x for _, x in sorted(zipped_pairs,reverse = True)]
        
        return best_recom[:3]    

In [28]:
print(bestrecommend.getContext_bestrecommend("doctor piyush clinic location"))

['what is the clinic location of doctor piyush', 'what is the clinic location of doctor soumya', 'what is the clinic location of doctor mayank']


In [29]:
print(bestrecommend.getContext_bestrecommend("aditya sickness reason"))

['what is the sickness reason for aditya sonkar', 'what is the reason of sickness of maximum age patient', 'what is the sickness reason for amit patel']


In [30]:
print(bestrecommend.getContext_bestrecommend("aditya  health issue"))

['what is the health issue with aditya sonkar', 'what is the health issue with amit', 'what is the health issue with yuvraj']


_______________________

## Database Connection 

In [31]:
import sqlite3
conn = sqlite3.connect("sql1.db")
cur = conn.cursor()


## Answer Desiging 

In [32]:
import sqlite3
class respond:
    def sql_answer(input_query):
        input_query = input_query.lower()
        conn = sqlite3.connect("sql1.db")
        cur = conn.cursor()
        if(input_query == "who are the best doctors for heart disease"):
            mycur = cur.execute("SELECT Doctor_Name FROM DOCTOR WHERE Speciality = 'Cardiologist' ")
            for i in mycur:
                print("{} is best doctor for heart disease".format(i[0]))
        elif(input_query == "who are the best doctors for cardiovascular disease"):
            mycur = cur.execute("SELECT Doctor_Name FROM DOCTOR WHERE Speciality = 'Cardiologist' ")
            for i in mycur:
                print("{} is the best doctor for cardiovasular disease".format(i[0]))
        elif(input_query == "who are the best doctors for skin"):
            mycur = cur.execute("SELECT Doctor_Name FROM DOCTOR WHERE Speciality = 'Dermatologist' ")
            for i in mycur:
                print("{} is the best doctor for skin".format(i[0]))
        elif(input_query == "best doctors for dermatology"):
            mycur = cur.execute("SELECT Doctor_Name FROM DOCTOR WHERE Speciality = 'Dermatologist' ")
            for i in mycur:
                print("{} is the best doctor for Dermatology".format(i[0]))
        elif(input_query == "best physician"):
            mycur = cur.execute("SELECT Doctor_Name FROM DOCTOR WHERE Speciality = 'Physician' ")
            for i in mycur:
                print("{} is the best Physician".format(i[0]))
        elif(input_query == "what is the clinic location of doctor mayank"):
            position = reversegeocoding.reversegeocoding_loc(input_query)
            print("Clinic location of Doctor Mayank is {}".format(position))
            
        elif(input_query == "what is the clinic location of doctor piyush"):
            position = reversegeocoding.reversegeocoding_loc(input_query)
            print("Clinic location of Doctor Piyush is {}".format(position))
            
        elif(input_query == "what is the clinic location of doctor soumya"):
            position = reversegeocoding.reversegeocoding_loc(input_query)
            print("Clinic location of Doctor Soumya is {}".format(position))   
            
        elif(input_query == "what is the clinic location of doctor deepshikha"):
            position = reversegeocoding.reversegeocoding_loc(input_query)
            print("Clinic location of Doctor Deepshikha is {}".format(position))   
            
            
        elif(input_query == "best doctor near my location for cardiology"):
            mycur = cur.execute("SELECT Doctor_Name FROM DOCTOR WHERE Speciality = 'Cardiologist' ")
            for i in mycur:
                print("Best Doctor for Cardiology is {}".format(i[0]))    
    
        elif(input_query == "Please provide the name of the doctors which have fees less than 500 rupees"):
            mycur = cur.execute("SELECT Doctor_Name FROM DOCTOR WHERE Fees <=500 ")
            for i in mycur:
                print("Doctor with fees less than aur equal to 500 is/are {}".format(i[0]))   
            
        elif(input_query == "what is timing for doctor soumya"):
            print("currently we are unavaiable to provide time schedule")
            
        elif(input_query == "which is the best dental clinic"):
            print("currently we have no data for dental clinic")
            
        elif(input_query == "what is the dr. rishabh consultant fees"):
            mycur = cur.execute("SELECT Fees FROM DOCTOR WHERE Doctor_Name = 'Rishabh Yadav'")
            for i in mycur:
                print("consultation charge of the Doctor Rishabh is {}".format(i[0]))    
        
        elif(input_query == "doctor with consultant fees more than 1000 rupees"):
            mycur = cur.execute("SELECT Doctor_Name FROM DOCTOR WHERE Fees >= 1000")
            for i in mycur:
                for j in range(len(i)):
                    print("Doctor with consultation charges more than 1000 rupees is {}".format(i[j]))
    
        elif(input_query == "what is the speciliatity of doctor piyush"):
            mycur = cur.execute("SELECT Speciality FROM DOCTOR WHERE Doctor_Name = 'Piyush Singh' ")
            for i in mycur:
                print("Specilization of Doctor Piyush is {}".fomat(i[0]))
                
        elif(input_query =="what is the rating of doctor piyush singh"):
            mycur = cur.execute("SELECT Rating FROM DOCTOR WHERE Doctor_Name = 'Piyush Singh' ")
            for i in mycur:
                print("Rating of Doctor Piyush is {}".format(i[0]))        
                
                
        elif(input_query == "what is the speciliatity of doctor mayank"):
            mycur = cur.execute("SELECT Speciality FROM DOCTOR WHERE Doctor_Name = 'Mayank Mishra' ")
            for i in mycur:
                print("Specilization of Doctor Mayank is {}".format(i[0]))
                
        elif(input_query == "what is the rating of doctor mayank mishra"):
            mycur = cur.execute("SELECT Rating FROM DOCTOR WHERE Doctor_Name = 'Mayank Mishra' ")
            for i in mycur:
                print("Rating of Doctor Mayank is {}".format(i[0]))        
                
                
        elif(input_query == "what is the speciliatity of doctor soumya"):
            mycur = cur.execute("SELECT Speciality FROM DOCTOR WHERE Doctor_Name = 'Soumya' ")
            for i in mycur:
                print("Specilization of Doctor Soumya is {}".format(i[0]))
                
        elif(input_query == "what is the rating of doctor soumya"):
            mycur = cur.execute("SELECT Rating FROM DOCTOR WHERE Doctor_Name = 'Soumya' ")
            for i in mycur:
                print("Rating of Doctor Soumya is {}".format(i[0]))        
                
                
        elif(input_query == "what is the speciliatity of doctor rishabh"):
            mycur = cur.execute("SELECT Speciality FROM DOCTOR WHERE Doctor_Name = 'Rishabh Yadav' ")
            for i in mycur:
                print("Specilization of Doctor Rishabh is {}" .format(i[0]))
                
        elif(input_query == "what is the rating of doctor rishabh yadav"):
            mycur = cur.execute("SELECT Rating FROM DOCTOR WHERE Doctor_Name = 'Rishabh Yadav'")
            for i in mycur:
                print("Rating of Doctor Rishabh is {}" .format(i[0]))        
                
                
        elif(input_query == "what is the speciliatity of doctor deepshikha"):
            mycur = cur.execute("SELECT Speciality FROM DOCTOR WHERE Doctor_Name = 'Deepshikha Mishra' ")
            for i in mycur:
                print("Specilization of Doctor Deepshikha is {}".format(i[0]))
                
        elif(input_query == "what is the rating of doctor deepshikha"):
            mycur = cur.execute("SELECT Rating FROM DOCTOR WHERE Doctor_Name = 'Deepshikha Mishra' ")
            for i in mycur:
                print("Rating of Doctor Deepshikha is {}".format(i[0]))        
                
                
        elif(input_query == "what is consultant fees of doctor piyush"):
            mycur = cur.execute("SELECT Fees FROM DOCTOR WHERE Doctor_Name = 'Piyush Singh' ")
            for i in mycur:
                print("consultation charges of Doctor Piyush is {}  Rupees".format(i[0]))         
                
                
        elif(input_query == "what is consultant fees of doctor mayank"):
            mycur = cur.execute("SELECT Fees FROM DOCTOR WHERE Doctor_Name = 'Mayank Mishra' ")
            for i in mycur:
                print("consultation charges of Doctor Mayank is {} Rupees" .format(i[0]))
                
                
        elif(input_query == "what is the consulatant fees of doctor soumya"):
            mycur = cur.execute("SELECT Fees FROM DOCTOR WHERE Doctor_Name = 'Soumya' ")
            for i in mycur:
                print("consultation charges of Doctor Soumya is {} Rupees".format(i[0]))  
                
                
        elif(input_query == "what is the consulatant fees of doctor deepshikha"):
            mycur = cur.execute("SELECT Fees FROM DOCTOR WHERE Doctor_Name = 'Deepshikha Mishra' ")
            for i in mycur:
                print("consultation charges of Doctor Deepshikha is {} Rupees".format(i[0]))
                
                
        elif(input_query == "who is the doctor of patient aditya"):
            mycur = cur.execute("SELECT Doctor_Name FROM Doctor WHERE Pat_id=(SELECT Patient_id FROM Patient WHERE Patient_Name = 'Aditya Sonkar')")
            for i in mycur:
                print("{} is the Doctor of patient Aditya".format(i[0]))
                
                
        elif(input_query == "who is the doctor of patient harsh"):
            mycur = cur.execute("SELECT Doctor_Name FROM Doctor WHERE Pat_id=(SELECT Patient_id FROM Patient WHERE Patient_Name = 'Harsh Mishra')")
            for i in mycur:
                print("{} is the Doctor of patient Harsh".format(i[0]))
                
                
        elif(input_query == "who is the doctor of patient yuvraj"):
            mycur = cur.execute("SELECT Doctor_Name FROM Doctor WHERE Pat_id=(SELECT Patient_id FROM Patient WHERE Patient_Name = 'Yuvraj Sinha')")
            for i in mycur:
                print("{} is the Doctor of patient Yuvraj".format(i[0]))
                
                
        elif(input_query == "who is the doctor of patient amit"):
            mycur = cur.execute("SELECT Doctor_Name FROM Doctor WHERE Pat_id=(SELECT Patient_id FROM Patient WHERE Patient_Name = 'Amit')")
            for i in mycur:
                print("{} is the Doctor of patient Amit".format(i[0]))
                
        elif(input_query == "who is the doctor of patient garima"):
            mycur = cur.execute("SELECT Doctor_Name FROM Doctor WHERE Pat_id=(SELECT Patient_id FROM Patient WHERE Patient_Name = 'Garima')")
            for i in mycur:
                print("{} is the Doctor of patient garima".format(i[0]))
                
                
        elif(input_query == "what is the minium age of the patient of doctor mayank"):
            mycur = cur.execute("SELECT min(Age) FROM Patient WHERE Patient_id =(SELECT Pat_id from Doctor WHERE Doctor_Name='Mayank Mishra')")
            for i in mycur:
                print("Minimum age of the Patient under Doctor Mayank Is {} years".format(i[0]))
                
                
        elif(input_query == "what is the sickness reason of devashree gupta"):
            print("Data for Devashree is not available")
            
        elif(input_query == "what is the sickness reason for aditya sonkar"):
            mycur = cur.execute("SELECT Issue from Patient WHERE Patient_Name = 'Aditya Sonkar' ")    
            for i in mycur:
                print("{} is the Health issue with patient Aditya sonkar".format(i[0]))
                
                      
        elif(input_query == "what is the sickness reason for amit patel"):
            mycur = cur.execute("SELECT Issue from Patient WHERE Patient_Name = 'Amit' ")    
            for i in mycur:
                print("{} is the health issue with patient Amit".format(i[0]))      
                
                
        elif(input_query == "how many girls are sick from last year"):
            mycur = cur.execute("SELECT count(Gender) FROM Patient WHERE Gender = 'F' ")    
            for i in mycur:
                print("{} is the total count  for sick girls".format(i[0]))
                
        elif(input_query == "what is the maximum age of the patient"):
            mycur = cur.execute("SELECT max(Age) FROM Patient ")    
            for i in mycur:
                print(i[0])        
       
        elif(input_query == "what is the reason of sickness of alok patel"):
            print("Currently No Data Available for the Patient Name Alok")
            
        elif(input_query == "most sick patiet"):
            mycur = cur.execute("SELECT Patient_Name FROM patient GROUP BY(Issue) HAVING Age = max(Age) AND Issue = 'Heart Attack'  ")    
            for i in mycur:
                print(i[0])     
            
        elif(input_query == "what is the age and gender of minimun age patient"):
            mycur = cur.execute("SELECT min(Age),Gender FROM Patient")    
            for i in mycur:
                print(i)
                
                
        elif(input_query == "what is the reason of sickness of maximum age patient"):
            mycur = cur.execute("SELECT Issue FROM Patient WHERE Patient_Name = (SELECT Patient_Name FROM Patient GROUP By(Age) HAVING Age=(Select max(Age) FROM Patient))")    
            for i in mycur:
                print(i[0])        
            
        elif(input_query == "what is the health issue with aditya sonkar"):
            mycur = cur.execute("SELECT Issue FROM Patient WHERE Patient_Name = 'Aditya Sonkar' ")    
            for i in mycur:
                print("{} is the Health issue with  Aditya Sonkar".format(i[0]))
        
        elif(input_query == "disease name of patient aditya sonkar"):
            mycur = cur.execute("SELECT Issue FROM Patient WHERE Patient_Name = 'Aditya Sonkar' ")    
            for i in mycur:
                print("{} is the Health issue with  Aditya Sonkar".format(i[0]))
  
    
    
    
                
        elif(input_query == "what is the health issue with harsh"):
            mycur = cur.execute("SELECT Issue FROM Patient WHERE Patient_Name = 'Harsh Mishra' ")    
            for i in mycur:
                print("{} is the Health issue with  Patient Harsh Mishra".format(i[0]))
                
        elif(input_query == "disease name of patient harsh mishra"):
            mycur = cur.execute("SELECT Issue FROM Patient WHERE Patient_Name = 'Harsh Mishra' ")    
            for i in mycur:
                print("{} is the Health issue with  Patient Harsh Mishra".format(i[0]))        
                
                
                
        elif(input_query == "what is the health issue with yuvraj"):
            mycur = cur.execute("SELECT Issue FROM Patient WHERE Patient_Name = 'Yuvraj Sinha' ")    
            for i in mycur:
                print("{} is the Health issue with  Patient Yuvraj Sinha".format(i[0]))
                
        elif(input_query == "disease name of patient yuvraj sinha"):
            mycur = cur.execute("SELECT Issue FROM Patient WHERE Patient_Name = 'Yuvraj Sinha' ")    
            for i in mycur:
                print("{} is the Health issue with  Patient Yuvraj Sinha".format(i[0]))        
                
                
        elif(input_query == "what is the health issue with amit"):
            mycur = cur.execute("SELECT Issue FROM Patient WHERE Patient_Name = 'Amit' ")    
            for i in mycur:
                print("{} is the Health issue with  Patient Amit".format(i[0]))
                
        
        elif(input_query == "disease name of patient amit"):
            mycur = cur.execute("SELECT Issue FROM Patient WHERE Patient_Name = 'Amit' ")    
            for i in mycur:
                print("{} is the Health issue with  Patient Amit".format(i[0]))
                
                
        elif(input_query == "what is the health issue with garima"):
            mycur = cur.execute("SELECT Issue FROM Patient WHERE Patient_Name = 'Garima' ")    
            for i in mycur:
                print("{} is the Health issue with  Patient Garima".format(i[0]))
                
        
        elif(input_query == "disease name of patient yuvraj sinha"):
            mycur = cur.execute("SELECT Issue FROM Patient WHERE Patient_Name = 'Garima' ")    
            for i in mycur:
                print("{} is the Health issue with  Patient Garima".format(i[0]))        
                
                
        elif(input_query == "maximum age patient name"):
            mycur = cur.execute("SELECT Patient_Name FROM Patient GROUP BY(Age) HAVING Age = (SELECT max(Age) FROM Patient) ")    
            for i in mycur:
                print(i[0])
                
        elif(input_query == "minimum age patient name"):
            mycur = cur.execute("SELECT Patient_Name FROM Patient GROUP BY(Age) HAVING Age = (SELECT min(Age) FROM Patient) ")    
            for i in mycur:
                print(i[0])
                
        elif(input_query == "average age of patients"):
            mycur = cur.execute("SELECT avg(Age) FROM Patient")    
            for i in mycur:
                print(i[0])
                
        elif(input_query == "best rating doctor"):
            mycur = cur.execute("SELECT Doctor_Name FROM Doctor WHERE Rating = (SELECT max(Rating) FROM Doctor)")    
            for i in mycur:
                for j in range(len(i)):
                    print("Highest rating Doctors are {}".format(i[j]))
                    
        elif(input_query == "worst rating doctor"):
            mycur = cur.execute("SELECT Doctor_Name FROM Doctor WHERE Rating = (SELECT min(Rating) FROM Doctor)")    
            for i in mycur:
                for j in range(len(i)):
                    print("lowest rating Doctors is {}".format(i[j]))   
                    
        elif(input_query == "highesst rating doctor"):
            mycur = cur.execute("SELECT Doctor_Name FROM Doctor WHERE Rating = (SELECT max(Rating) FROM Doctor)")    
            for i in mycur:
                for j in range(len(i)):
                    print("Highest rating Doctors are {}".format(i[j]))
                    
        elif(input_query == "lowest rating doctor"):
            mycur = cur.execute("SELECT Doctor_Name FROM Doctor WHERE Rating = (SELECT min(Rating) FROM Doctor)")    
            for i in mycur:
                for j in range(len(i)):
                    print("lowest rating Doctors is {}".format(i[j]))            
                
                
            

In [41]:
respond.sql_answer("what is the consulatant fees of doctor soumya")



consultation charges of Doctor Soumya is 600 Rupees


In [34]:
import reverse_geocoder as rf

class reversegeocoding:
    def reversegeocoding_loc(input_query):
        con = sqlite3.connect("sql1.db")
        cur = con.cursor()
        
        if(input_query == "what is the clinic location of doctor mayank"):
            mycur = cur.execute("SELECT Clinic_lat,Clinic_long FROM Doctor WHERE Doctor_Name='Mayank Mishra' ")
            for i in mycur:
                lat = i[0]
                long = i[1]
            latitude = float(lat)
            longititude = float(long)
            coordinates = (latitude,longititude)
            location = rf.search(coordinates)
           
            
        elif(input_query == "what is the clinic location of doctor piyush"):
            mycur = cur.execute("SELECT Clinic_lat,Clinic_long FROM Doctor WHERE Doctor_Name='Piyush Singh' ")
            for i in mycur:
                lat = i[0]
                long = i[1]
            latitude = float(lat)
            longititude = float(long)
            coordinates = (latitude,longititude)
            location = rf.search(coordinates)
        
            
        elif(input_query == "what is the clinic location of doctor soumya"):
            mycur = cur.execute("SELECT Clinic_lat,Clinic_long FROM Doctor WHERE Doctor_Name='Soumya' ")
            for i in mycur:
                lat = i[0]
                long = i[1]
            latitude = float(lat)
            longititude = float(long)
            coordinates = (latitude,longititude)
            location = rf.search(coordinates)
           
            
        elif(input_query == "what is the clinic location of doctor deepshikha"):
            mycur = cur.execute("SELECT Clinic_lat,Clinic_long FROM Doctor WHERE Doctor_Name='Deepshikha Mishra' ")
            for i in mycur:
                lat = i[0]
                long = i[1]
            latitude = float(lat)
            longititude = float(long)
            coordinates = (latitude,longititude)
            location = rf.search(coordinates)
        
        
        elif(input_query == "what is the clinic location of doctor rishabh"):
            mycur = cur.execute("SELECT Clinic_lat,Clinic_long FROM Doctor WHERE Doctor_Name='Rishabh Yadav' ")
            for i in mycur:
                lat = i[0]
                long = i[1]
            latitude = float(lat)
            longititude = float(long)
            coordinates = (latitude,longititude)
            location = rf.search(coordinates)

        return location[0]['name'] 
    

In [35]:
k = reversegeocoding.reversegeocoding_loc("what is the clinic location of doctor rishabh")
g = reversegeocoding.reversegeocoding_loc("what is the clinic location of doctor mayank")
print(g)
print(k)

Loading formatted geocoded file...
Gurgaon
Vijayawada


In [36]:
Doctors = ["Piyush Singh","Soumya","Mayank Mishra","Deepshikha Mishra","Rishabh Yadav"]

In [37]:
Patients = ["Aditya Sonkar","Harsh Mishra","Yuvraj Sinha","Amit","Garima"]

In [42]:
speciality = ["Cardiologist","Physician","Dermatologist","Surgeon","Psycheatrist"]

## Chatbot desigining

In [38]:
flag = True
context = None
Bestmatch = None
reply = None

In [39]:
while flag:
    Yes_check = 'Yes'
    No_check = 'No'
    query =input("User >")
    query = query.strip()
    query = query.lstrip("@,<,>,#,$,%,%,^,&,*,?")
    query = query.rstrip("@,<,>,#,$,%,%,^,&,*,?")
    query = regrex(query)
    print("Bot > Hello, I am a bot design for Specific Purpose How can I help you?")
    time.sleep(1)
    print("Bot > I can help you to provide information about Doctor Speciliaty,Doctor Fees,Clinic Location,Doctor Rating")
    time.sleep(1)
    print("Bot > I can also help you to provide the information about the Patient Health Issue,Patient Doctor")
    time.sleep(1)
    print("Bot > Here is the list of Doctors {}".format(Doctors))
    time.sleep(1)
    print("Bot > Here is the list of Patients {}".format(Patients))
    time.sleep(1)
    print("Bot > Please Provide your Query for Search")
    query = input("User >")
    query = regrex(query)

    score,_,context = getContext_details(query)
    
    if len(query)==0:
        print("Bot > Please provide the Input")
    elif query=="exit":
        flag=False
    
    else:
        if(score<0.4):
            print("Bot > I am a bot design for Database related issues, Please Provide Suitable Query")
            
        elif(score>=1.0):
            _,best_match,_ = getContext_details(query)
            answer = respond.sql_answer(query)
            print("Bot > Thanks for Searching")
            
            
            
            
        elif(score>=0.4 and context == "Doctor"):
            print("Bot > You want to check Detail Information about Doctor {} Yes or No" .format(get_name1((query))))
            query =input("User > ")
            query = query.lstrip("@,<,>,#,$,%,%,^,&,*,?")
            query = query.rstrip("@,<,>,#,$,%,%,^,&,*")
            query = regrex(query)
            if(query == "exit"):
                flag = False
            if (query.casefold()==Yes_check.casefold()):
                print("Bot > Please provide your search query again for confirmation")
                query =input("User >")
                query = query.lstrip("@,<,>,#,$,%,%,^,&,*,?")
                query = query.rstrip("@,<,>,#,$,%,%,^,&,*")
                query = regrex(query)
                if(query == "exit"):
                    flag = False
                else:
                    Bestmatch = bestrecommend.getContext_bestrecommend(query)
                    for response in Bestmatch:
                        print("Bot > {} ".format(response))
                        print(" ")
                    print("Bot > None of the Above")
                    print(" ")
                    print("Bot > Above are the matching query,Select the suitable query")
                    print("Bot > Enter the best suitable query")
                    query = input("User >")
                    query = query.lstrip()
                    query = query.lstrip("@,<,>,#,$,%,%,^,&,*")
                    query = regrex(query)
                    if (query=="exit"):
                        flag=False
                    elif(query == "None of the Above"):
                        print("Bot > I am unable to  find this query this time,I am Design for Specific Purpose, I will help you after some time")
                        break
                    else:
                        respond.sql_answer(query)
                        time.sleep(2)
                        
                
                
            else:
                print("bot > Sorry, I am currently unavailable to find query")
                flag = False
                
        elif(score>=0.4 and context=="Patient"):
            print("Bot > Paient Informations are very Sensitive Information to check the information \n Please Provide the Patient id")
            time.sleep(2)
            
            query = input("User > ")
            query = query.lstrip("@,<,>,#,$,%,%,^,&,*,?")
            query = query.rstrip("@,<,>,#,$,%,%,^,&,*,?")
            try:
                query = int(query)
                print("Bot > You want to check Detail Information about Patient {} Yes or No" .format(patient_name(query)))   
                query = input("User >")
                query = regrex(query)
                if (query=="exit"):
                    flag = False
                if (query.casefold()==Yes_check.casefold()):
                    print("Bot > Please provide your Search query again for confirmation")
                    query = input("User >")
                    query = regrex(query)
                    if (query == "exit"):
                        flag = False
                    else:
                        Bestmatch_1 = bestrecommend.getContext_bestrecommend(query)
                        for response in Bestmatch_1:
                            print("Bot > {}".format(response))
                            print(" ")
                        print("Bot > None of the Above") 
                        print(" ")
                        print("Bot > Above are the matching query,Select the suitable query")
                        print("Bot > Enter the best suitable query")
                        query = input("User >")
                        query = query.lstrip()
                        query = query.lstrip("@,<,>,#,$,%,%,^,&,*,?")
                        query = regrex(query)
                        if (query=="exit"):
                            flag = False
                    
                        elif(query == "None of the Above"):
                            print("Bot > I am unable to  find this query this time,I am Design for Specific Purpose, I will help you after some time")
                            break
                    
                        else:
                            answer = respond.sql_answer(query)
                            print("Please provide query to search again or type exit to end the Conversation")    
                
              
                elif (query.casefold()==No_check.casefold()):
                    print("bot > Sorry, I am currently unavailable to find query")
                    print("Please provide query to search again or type exit to end the Conversation")
            except:
                print("Bot > You are providing incorrect input")
                
                
            

    

User > hi


Bot > Hello, I am a bot design for Specific Purpose How can I help you?
Bot > I can help you to provide information about Doctor Speciliaty,Doctor Fees,Clinic Location,Doctor Rating
Bot > I can also help you to provide the information about the Patient Health Issue,Patient Doctor
Bot > Here is the list of Doctors ['Piyush Singh', 'Soumya', 'Mayank Mishra', 'Deepshikha Mishra', 'Rishabh Yadav']
Bot > Here is the list of Patients ['Aditya Sonkar', 'Harsh Mishra', 'Yuvraj Sinha', 'Amit', 'Garima']
Bot > Please Provide your Query for Search


User > what is the consultant fees of doctor soumya


Bot > Thanks for Searching


User > 


Bot > Hello, I am a bot design for Specific Purpose How can I help you?
Bot > I can help you to provide information about Doctor Speciliaty,Doctor Fees,Clinic Location,Doctor Rating
Bot > I can also help you to provide the information about the Patient Health Issue,Patient Doctor
Bot > Here is the list of Doctors ['Piyush Singh', 'Soumya', 'Mayank Mishra', 'Deepshikha Mishra', 'Rishabh Yadav']
Bot > Here is the list of Patients ['Aditya Sonkar', 'Harsh Mishra', 'Yuvraj Sinha', 'Amit', 'Garima']
Bot > Please Provide your Query for Search


User > exit


In [59]:
query = input("")
query = query.split()
for i in query:
    for j in doctors:
        if i==j:
            name = j
            print(name)
            

 piyush singh


piyush


In [61]:

name  = "Piyush Singh"
cur.execute('SELECT Fees FROM Doctor WHERE Doctor_Name=?',(name,))
row = cur.fetchone()
print(row)

for k in cur:
    print(k[0])
            
            
        

(1000,)
